In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import re

from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd

from wordcloud import *
from PIL import Image
import csv
import random

from gensim import *
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import *

%matplotlib inline

D:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#os.chdir("D:/grad/MA thesis/data/meta stat")
sns.set(style = 'whitegrid')

#basic setting: colour
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)    

<font size = "4">Figure 1(a) - Figure 2: Metadata</fontsize> 

Figure 1(a): user creation date. Note this is a version suitable for Latex output. 

In [3]:
#loading data
users = pd.read_csv("user_created_at.csv")
user_slice = users.loc[3442:] #may be edited for other time slices
user_slice["formatted"] = [datetime.strptime(d, "%Y/%m/%d") for d in user_slice["DATE(user_created_at_formatted)"]]
rolling = user_slice["COUNT(*)"].rolling(window=30)
rolling_mean = rolling.mean()
user_slice["monthly"] = rolling_mean

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [ ]:
fig = plt.figure(figsize = (20,8))
ax1 = fig.add_subplot(111)

ax1.plot(user_slice['formatted'], user_slice['COUNT(*)'], label = "Daily sum", color = tableau20[0])
ax1.plot(user_slice["formatted"], user_slice["monthly"],label = "Monthly rolling average",  
         color = tableau20[2])
ax1.legend(prop={'size': 15})
ax1.grid(linestyle = 'dashed')
ax1.spines["top"].set_visible(False)       
ax1.spines["right"].set_visible(False)  

plt.grid(linestyle = 'dashed')
ax1.set_ylabel('Newly registered users',fontsize = 20)
#setting time axis
yearsFmt = mdates.DateFormatter('%Y-%m')
ax1.xaxis.set_major_formatter(yearsFmt)
ax1.set_xlabel('Date',fontsize = 20)
#providing directions for some key incidents
times = ["2016-06-23","2017-01-17","2018-05-27"]
for t in times:
    ax1.axvline(x = t, linewidth = 1, linestyle = "dashed")

ax1.axvspan('2019-3-1', '2019-5-4', alpha=0.1, color='grey')
ax1.axvspan('2018-11-13', '2018-12-10', alpha=0.1, color='grey')
ax1.text(x = "2016-06-18", y = 320, horizontalalignment = 'right', s = "Brexit\nreferendum", fontsize = 15)
ax1.text(x = "2017-01-15", y = 400, horizontalalignment = 'right', s = "Theresa May's\nBrexit speech",
        fontsize = 15)
ax1.text(x = "2018-05-20", y = 400, horizontalalignment = 'right',s = "Parliment passed\nwithdrawl act",
       fontsize = 15)
ax1.text(x = "2018-11-30", y = 500, horizontalalignment = 'right',s = "Period before the\n(postponed) deal vote",
        fontsize = 15)
ax1.text(x = "2019-03-08", y = 700, horizontalalignment = 'right', s = "EU parliament election\npropoganda",
               fontsize = 15)

#plt.savefig("meta_1.png")
#plt.savefig("meta_1.pdf")
plt.show()

Meta data figure 1(b): daily number of tweets.

In [5]:
#loading data
tweets = pd.read_csv("number of tweets.csv")
tweet_rolling = tweets["Intrapolated"].rolling(window=7)
tweet_rolling_mean = tweet_rolling.mean()
tweets["weekly"] = tweet_rolling_mean
tweets["formatted"] = [datetime.strptime(d, "%d/%m/%Y") for d in tweets["DATE(created_at_formatted)"]]

In [ ]:
#plotting
fig = plt.figure(figsize = (20,8))
ax2 = fig.add_subplot(111)

dates = list(tweets["formatted"])
tweet_num = list(tweets["Intrapolated"])


ax2.bar(tweets["formatted"].values, tweets["Intrapolated"].values, label = "Daily sum")
ax2.plot(tweets["formatted"].values, tweet_rolling_mean ,label = "Weekly rolling average", color = tableau20[2])
ax2.legend(prop={'size': 15})
ax2.grid(linestyle = 'dashed')
ax2.spines["top"].set_visible(False)       
ax2.spines["right"].set_visible(False)  

ax2.set_xlabel('Date', fontsize = 20)
ax2.set_ylabel('Number of tweets scraped', fontsize = 20)
#plt.savefig("meta_2.png")
#plt.savefig("meta_2.pdf")
plt.show()


Figure 1(c): Bigram, with the tags manually tweaked a little bit...(selected the top 25 and deleted a meaningless one)

In [ ]:
#Data preprocessing
bigram_sample = pd.read_csv("sample_new.csv", encoding = 'latin-1')

with open("D:/grad/MA thesis/data/stopwords/smart-common-words.txt", 'r') as f:
    stopwords = f.read().split(",")

def preprocess(text,stopwords):
    result=[]
    text = text.lower()
    for token in utils.simple_preprocess(text) :
        if token not in stopwords and len(token) > 3:
            result.append(token)
    return result

def bigram_finder(data,stopwords):
    tag_counter = {}
    for i in data:
        tag = preprocess(i, stopwords)
        for t in tag:
            try:
                t = t.lower()
                if t in tag_counter:
                    tag_counter[t]  += 1
                else:
                    tag_counter[t] = 1
            except:
                pass
    return tag_counter

leave_tags = bigram_finder(bigram_sample[bigram_sample["leave_remain"]>0]["full_text"], stopwords)
remain_tags = bigram_finder(bigram_sample[bigram_sample["leave_remain"]<0]["full_text"], stopwords)

leave_top = {k: v for k, v in leave_tags.items() if v > 380}
remain_top = {k: v for k, v in remain_tags.items() if v > 430}

In [ ]:
ax3.barh(range(len(leave_top)), [i*20 for i in leave_top.values()], align='center', color = tableau20[0])  
ax3.barh(range(len(remain_top)), [-i*20 for i in remain_top.values()], align='center', color = tableau20[6])
ax3.set_xlabel('Phrase frequency', fontsize = 20)

rects = ax3.patches
leave_labels = list(leave_top.keys())
remain_labels = list(remain_top.keys())
# For each bar: Place a label

i = 0
for rect in rects:
    x_value = rect.get_width()
    y_value = rect.get_y() + rect.get_height() / 2
    space = 5
    ha = 'left'
    if x_value > 0:
        label = leave_labels[i]
    else:
        space *= -1
        ha = 'right'
        label = remain_labels[i - 24]
    ax3.annotate(
        label,                      # Use `label` as label
        (x_value, y_value),         # Place label at end of the bar
        xytext=(space, 0),          # Horizontally shift label by `space`
        textcoords="offset points", # Interpret `xytext` as offset in points
        va='center',                # Vertically center label
        ha=ha,
        fontsize = 15)
    i += 1                      # Horizontally align label differently for

ax3.get_yaxis().set_visible(False)
ax3.grid(False)
ax3.spines['right'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.spines['top'].set_visible(False)
ticks =  ax3.get_xticks()
ax3.set_xticklabels([int(abs(tick)) for tick in ticks])
ax3.legend(prop={'size': 15})
#plt.savefig("meta_3.png")
#plt.savefig("meta_3.pdf")
plt.show()

Figure 2: Choropleth of all users with reported location in UK, breakdown by adminstrative constitutencies. 
Imported processed .shp file directly. Please see geocoding.py for the codes.

In [ ]:
uk_shp = gpd.read_file("./geocoding/GBP_new.shp")
uk_shp["log_num"] = [math.log10(i) if i != 0 else 0 for i in uk_shp["real_point"] ]
uk_shp_nolondon = uk_shp.drop(index = 35)
london = uk_shp.iloc[35,:]

uk_shp.crs = {'init' :'epsg:4326'}
uk_shp = uk_shp.to_crs({'init': 'epsg:3395'})
fig = plt.figure(figsize = (20,20))
ax4 = fig.add_subplot(111)
ax4.set_aspect('equal')
ax4.set_ylim([6340000, 8550000])
ax4.set_xlim([-1200000, 282411])
uk_shp.plot(ax = ax4, color = 'white', edgecolor = "black")
divider = make_axes_locatable(ax4)
cax = divider.append_axes("left", size="8%", pad=0.1)
cax.tick_params(labelsize=20)
shp = uk_shp.plot(column = 'log_num',ax = ax4,  cmap = 'Blues',legend = True,cax = cax)
ax4.axis("off")

#plt.savefig("meta_4.pdf")
#plt.savefig("meta_4.png")
plt.show()


<font size = "5">Figure 3: Wordclouds</font size> 
- Figure 3(a): Hashtags

In [ ]:

def pattern_finder(data, regexp, tag_counter):
    for i in data:
        tag = re.findall(regexp, i)
        if tag is None:
            continue
        else:
            for t in tag:
                t = t.lower()
                t = t[1:]
                if t in tag_counter:
                    tag_counter[t]  += 1
                else:
                    tag_counter[t] = 1
    return tag_counter
'''
This part is from the example from wordcloud package document:
https://amueller.github.io/word_cloud/auto_examples/
    colored_by_group.html#sphx-glr-auto-examples-colored-by-group-py
'''
class SimpleGroupedColorFunc(object):
    def __init__(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def __call__(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)


with open('sample.csv', newline='', encoding = 'utf-8') as csvfile:
     sample = csv.reader(csvfile)
     tag_counter = {}
     corpus = ""
     for row in sample:
         tag_counter = pattern_finder(row, r'([#][\w_-]+)',tag_counter)
         

''' the hashtag cloud'''
#tag tendency is a preprocessed file with tag freqency & tag tendency(leave/remain). Please see other codes for the processing process.
tag_tendency = pd.read_csv("tag_label.csv", encoding = 'latin-1')
colour = {"red":[], "blue":[]}

for i in range(len(tag_tendency)):
    if tag_tendency["tendency"].iloc[i] == 1: 
        colour["red"].append(tag_tendency["tag"].iloc[i])
    elif tag_tendency["tendency"].iloc[i] == 3:
        colour["blue"].append(tag_tendency["tag"].iloc[i])
    else:
        pass
default_colour = 'grey'

mask = np.array(Image.open('uk1.png'))                 
wc = WordCloud(background_color="white", max_words = 400, mask = mask, 
                   width=1600, height = 800)
wc.generate_from_frequencies(tag_counter)
grouped_color_func = SimpleGroupedColorFunc(colour, default_colour)
wc.recolor(color_func=grouped_color_func)
plt.imshow(wc, interpolation="spline16")
plt.axis("off")
#wc.to_file("hashtag_cloud.png")
plt.show()

Figure 3(b): Leave and remain tweet keywords

In [ ]:
with open('stopwords.txt', 'r') as f:
    stopword = f.read()
stopwords = stopword.split(",")

def wc_generator(text_path, color_func, mask, savename, stopwords = stopwords, max_words = 400):
    wc = WordCloud(background_color="white", max_words = max_words, mask = mask, 
                   width=1600, height = 800, stopwords = stopwords
                   )
    text =  open(text_path, encoding = 'utf-8').read()
    wc.generate(text)
    plt.imshow(wc.recolor(color_func=color_func, random_state=3), interpolation="spline16")
    plt.axis("off")
    wc.to_file(savename)
    return

def red_color_func(word, font_size, position, orientation, random_state=None,
                    **kwargs):
    return "hsl(10, 100%%, %d%%)" % random.randint(40, 100)

def blue_color_func(word, font_size, position, orientation, random_state=None,
                    **kwargs):
    return "hsl(242, 100%%, %d%%)" % random.randint(40, 100)

#remain 
mask = np.array(Image.open('half_circle_left.jpg'))
mask_right = np.array(Image.open('half_circle_right.jpg'))
wc_generator('remain_text.txt', red_color_func, mask, 'remain_cloud.png')
#leave
wc_generator('leave_text.txt', blue_color_func, mask_right, 'leave_cloud.png')



<font size = "5">Figure 4(a) - 4(d): Sentiment</font size> 
- 4(a): Daily leave/remain tendency
- 4(b): Daily avg sentiment of leave/remain tweets
- 4(c): Sentiment of remain media + tweets
- 4(d): Sentiment of leave media + tweets

In [ ]:
#preprocessed data
twitter = pd.read_csv("leave_sentiment_count.csv")
newspaper = pd.read_csv("dailymail_senti.csv")

#Count avg daily tendency 
twitter["formatted"] = [datetime.strptime(i, "%Y/%m/%d") for i in twitter["date"]]
newspaper["formatted"] = [datetime.strptime(i, "%Y/%m/%d") for i in newspaper["date"]]
twitter = twitter.interpolate()
newspaper = newspaper.interpolate()
twitter = twitter.drop(index = [i for i in range(208,213)])
rolling = newspaper["guard_senti"].rolling(window=3)
rolling_mean = rolling.mean()

In [ ]:
fig = plt.figure(figsize = (20,7))
#fig 4(a) with average leave/remain tendency
axs1 = fig.add_subplot(111)
axs1.plot(twitter["formatted"], twitter["avg_leave_score"].values, label = "Daily pro-leave score")
axs1.legend(prop={'size': 15})
axs1.set_ylabel("Pro-leave tendency", fontsize = 20)
axs1.set_xlabel("Date", fontsize = 20)
axs1.axhline(linewidth = 2, color = "black")
axs1.grid(False)
axs1.set_xlim(['2018-11-13', '2019-06-04'])
axs1.spines["top"].set_visible(False)       
axs1.spines["right"].set_visible(False)  
plt.savefig("sentiment_1.pdf")


In [ ]:
#fig 4(b): sentiment scores
fig = plt.figure(figsize = (20,7))
axs2 = fig.add_subplot(111)
axs2.plot(twitter["formatted"], twitter["remain_senti"].values, label = "Sentiment of pro-remain tweets")
axs2.plot(twitter["formatted"], twitter["leave_senti"].values, label = "Sentiment of pro-leave tweets")
axs2.set_ylabel("Sentiment level", fontsize = 20)
axs2.set_xlabel("Date", fontsize = 20)
axs2.legend(prop={'size': 15})
axs2.grid(False)
axs2.set_xlim(['2018-11-13', '2019-06-04'])
axs2.spines["top"].set_visible(False)       
axs2.spines["right"].set_visible(False)  
plt.savefig("sentiment_2.pdf")


In [ ]:
#fig 4(c)
fig = plt.figure(figsize = (20,7))
axs3 = fig.add_subplot(111)
line1 = axs3.plot(twitter["formatted"], twitter["remain_senti"].values, label = "Sentiment of pro-remain tweets", 
   color = tableau20[0])
ax3_twin = axs3.twinx()
line2 = ax3_twin.plot(twitter["formatted"], newspaper["guard_senti"], label = "Sentiment of pro-remain media", color = tableau20[2])

set_legend = line1+line2
labs = [line.get_label() for line in set_legend]
axs3.legend(set_legend, labs, loc=3, prop={'size': 15})

axs3.set_ylabel("Sentiment level",fontsize = 20)
axs3.set_xlabel("Date", fontsize = 20)
ax3_twin.set_ylabel("Sentiment level",fontsize = 20)
plt.xlim('2018-11-13', '2019-06-04')
axs3.spines["top"].set_visible(False)
ax3_twin.spines["top"].set_visible(False)

plt.savefig("sentiment_3.pdf")

In [ ]:
#fig 4(d)
fig = plt.figure(figsize = (20,7))
axs4 = fig.add_subplot(111)
line1 = axs4.plot(twitter["formatted"], twitter["leave_senti"].values, label = "Sentiment of pro-leave tweets",
   color = tableau20[0])
ax4_twin = axs4.twinx()
line2 = ax4_twin.plot(twitter["formatted"], newspaper["mail_senti"].values, label = "Sentiment of pro-leave media", color = tableau20[2])
axs4.set_ylabel("Sentiment level",fontsize = 20)
ax4_twin.set_ylabel("Sentiment level",fontsize = 20)

set_legend = line1+line2
labs = [line.get_label() for line in set_legend]
axs4.legend(set_legend, labs, loc=4, prop={'size': 15})
axs4.spines["top"].set_visible(False)
ax4_twin.spines["top"].set_visible(False)

axs4.set_xlabel("Date", fontsize = 20)
plt.xlim('2018-11-13', '2019-06-04')
plt.savefig("sentiment_4.pdf")


<font size = "5">Figure 5 & 6: Topic modelling results, daily average proprotion of topics</font size> 

In [ ]:
#Uncomment for corpus with all tweets. 

#leave = pd.read_csv("leave_topicmodel.csv", usecols = [1,2],names = ['date','topic'])
leave = pd.read_csv("leave_topicmodel_original.csv", usecols = [1,2])    
leave['formatted'] = [datetime.strptime(i, "%Y-%m-%d %H:%M:%S") for i in leave["date"]]
leave.set_index('formatted', inplace = True)
leave['counter'] = [1] * len(leave['topic'])
counts = leave['counter'].groupby(pd.Grouper(freq = 'D')).count()

leave_calc = pd.DataFrame()
leave_calc['counts'] = counts
#for i in range(4):
for i in range(5):
    varname = 'topic_' + str(i)
    rolling_name = 'rolling_' + str(i)
    count = leave[leave['topic'] == i]['counter'].groupby(pd.Grouper(freq = 'D')).count()
    if len(count) != len(counts):
        count = pd.concat([pd.Series([0]), count])
    percent = [count[k]/counts[k] for k in range(len(count))]
    leave_calc[varname] = percent
    rolling = leave_calc[varname].rolling(window=3)
    rolling_mean = rolling.mean()
    leave_calc[rolling_name] = rolling_mean

leave_calc = leave_calc.interpolate()
#leave_calc.to_csv("topic_model_freq_leave_original.csv")


#remain = pd.read_csv("remain_topicmodel.csv", usecols = [1,2], names = ['date','topic'])
remain = pd.read_csv("remain_topicmodel_original.csv", usecols = [1,2], names = ['date','topic'])
remain['formatted'] = [datetime.strptime(i, "%Y-%m-%d %H:%M:%S") for i in remain["date"]]
remain.set_index('formatted', inplace = True)
remain['counter'] = [1] * len(remain['date'])
counts = remain['counter'].groupby(pd.Grouper(freq = 'D')).count()

remain_calc = pd.DataFrame()
remain_calc['counts'] = counts
#for i in range(4):
for i in range(3):
    varname = 'topic_' + str(i)
    rolling_name = 'rolling_' + str(i)
    count = remain[remain['topic'] == i]['counter'].groupby(pd.Grouper(freq = 'D')).count()
    percent = [count[k]/counts[k] for k in range(len(count))]
    remain_calc[varname] = percent
    rolling = remain_calc[varname].rolling(window=3)
    rolling_mean = rolling.mean()
    remain_calc[rolling_name] = rolling_mean
remain_calc = remain_calc.interpolate()

In [ ]:
#Fig 5: all tweets

counter = 0
plt.figure(figsize = (20,16))
ax = plt.subplot(211)    

ax.spines["top"].set_visible(False)       
ax.spines["right"].set_visible(False)  

plt.grid(linestyle = 'dashed')  


labels = ['Topic 1: The UK Indepedence Party',
          'Topic 2: Brexit Party \nfor EU election',
          'Topic 3: Labours for leave',
          'Topic 4: Conservative for leave',
          ]

for i in leave_calc.columns.values:
    if 'topic' in i:
        ax.plot(leave_calc.index.values, leave_calc[i].values, color = tableau20[counter],
                 label = labels[counter])
        counter += 1

plt.xlim('2018-11-17', '2019-06-10')
plt.legend(prop={'size': 13})
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

ax2 = plt.subplot(212)
counter = 0
ax2.spines["top"].set_visible(False)       
ax2.spines["right"].set_visible(False)  


plt.grid(linestyle = 'dashed')  

labels_remain = ["Topic 1: Revoke article 50",
                 "Topic 2: The liberals for remain\n and EU election",
                 "Topic 3: People's Vote",
                 "Topic 4: Brexit shambles"]

for i in remain_calc.columns.values:
    if 'rolling' in i:
        ax2.plot(remain_calc.index.values, remain_calc[i].values, color = tableau20[counter],
                 label = labels_remain[counter])
        counter += 1

plt.legend(prop={'size': 13})
plt.xlim('2018-11-17', '2019-06-10')
plt.xlabel("Date", fontsize = 20)
ax.set_ylabel("Prob(leave topics)", fontsize = 20)
ax2.set_ylabel("Prob(remain topics)", fontsize = 20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
#plt.figtext(0.1, 0.01, '''Source: Self-collected twitter data.
#            Figures were three-day rolling average; 
#            missing data points interpolated.''', ha='left', va='bottom') #Uncomment for .png output. This part was in .tex file.

#plt.savefig('topic_model_refined.pdf')
plt.show()


In [ ]:
#Figure 6: original only
counter = 0
plt.figure(figsize = (20,16))
ax = plt.subplot(211)    

ax.spines["top"].set_visible(False)       
ax.spines["right"].set_visible(False)  

plt.grid(linestyle = 'dashed')  


labels = ['Topic 1: UKIP & Brexit Party \n EU election',
          'Topic 2: Brexit politicians \n BBC politics',
          'Topic 3: Labour for leave',
          'Topic 4: Attacking the remainers',
          'Topic 5: Right-wing opinion \nand memes']


for i in leave_calc.columns.values:
    if 'topic' in i:
        ax.plot(leave_calc.index.values, leave_calc[i].values, color = tableau20[counter],
                 label = labels[counter])
        counter += 1

plt.xlim('2018-11-17', '2019-06-10')
plt.legend(prop={'size': 13})
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

ax2 = plt.subplot(212)
counter = 0
ax2.spines["top"].set_visible(False)       
ax2.spines["right"].set_visible(False)  


plt.grid(linestyle = 'dashed')  

labels_remain = ["Topic 1: People's Vote",
                 "Topic 2: Revoke article 50",
                 'Topic 3: Brexit shambles']

for i in remain_calc.columns.values:
    if 'rolling' in i:
        ax2.plot(remain_calc.index.values, remain_calc[i].values, color = tableau20[counter],
                 label = labels_remain[counter])

        counter += 1

plt.legend(prop={'size': 13})
plt.xlim('2018-11-17', '2019-06-10')
plt.xlabel("Date", fontsize = 20)
ax.set_ylabel("Prob(leave topics)", fontsize = 20)
ax2.set_ylabel("Prob(remain topics)", fontsize = 20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

#plt.savefig('topic_model_refined_all.pdf')
plt.show()


<font size = "5">Figure 7 & 8: ML metrics </font size> 

In [ ]:
#Figure 7: ML metrics (F1)
f1 = {'Baseline':0.789,
      'Logistic Regression':0.817,
      'VADER':0.789,
      'Random Forest': 0.940}


plt.figure()
ax = plt.subplot(111)
ax.bar(range(len(f1)),list(f1.values()),align = 'center', width = 0.5, alpha=0.6)
plt.xticks(range(len(f1)), list(f1.keys()))
ax.set_ylabel('F1 Score')
ax.set_xlabel('Methods')
ax.spines["top"].set_visible(False)     ,  
ax.spines["right"].set_visible(False)  
for i, v in enumerate(list(f1.values())):
    ax.text(i - .14 , v+.02,str(v), color='black')

plt.show()

In [ ]:
#Figure 8: topic model u_mass coherence
metrics = pd.read_excel(r"metrics.xlsx")
metrics["topics"] = [i for i in range(2,14)]

fig, axs = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
axs[0].plot(metrics['topics'], metrics['twitter-leave-umass'])
axs[0].set_xlabel('Number of topics(leave - all)', fontsize = 14)
axs[0].set_ylabel('u_mass coherence', fontsize = 14)
axs[1].plot(metrics['topics'], metrics['twitter-remain'])
axs[1].set_xlabel("Number of topics(remain - all)", fontsize = 14)
axs[0].spines["top"].set_visible(False)      
axs[1].spines["top"].set_visible(False)      
axs[0].spines["right"].set_visible(False)   
axs[1].spines["right"].set_visible(False)  

plt.show()
